In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/dSph/draco/netflow/TEST/draco_6_1_021/ga-netflow_20250320085318.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/draco/netflow/TEST/draco_6_1_021/'

# Plot the design files

In [0]:
import os, sys, re
from glob import glob
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils
from pfs.datamodel import PfsDesign, FiberStatus, TargetType

from pfs.ga.targeting.scripts.netflow.notebooks.notebooks import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.util.astro import *
from pfs.ga.targeting.io import GaiaReader

# Load the netflow config file

In [0]:
# Load the configuration
config = load_netflow_config(CONFIG_FILE)

# Plot the design files

In [0]:
# Create the PFI instrument. This will load the instrument calibration data.
pfi = SubaruPFI(instrument_options=config.instrument_options)

In [0]:
# Find and load the design files in the output directory
designs = {}
design_files = glob(os.path.join(OUTPUT_PATH, 'pfsDesign*.fits'))
for fn in design_files:
    pfsDesignId = int(re.search(r'0x[0-9a-f]+', fn).group(0), 16)
    design = PfsDesign.read(pfsDesignId, dirName=os.path.dirname(fn))
    designs[pfsDesignId] = design
    print(f'Loaded design 0x{pfsDesignId:016x}')

In [0]:
# Verify that there are no duplicate fiberIds
for pfsDesignId, design in designs.items():
    counts = np.bincount(design.fiberId)
    print(f'0x{pfsDesignId:016x}', np.where(counts > 1))

In [0]:
cmap = plt.get_cmap('tab10')

for pfsDesignId, design in designs.items():

        f = plt.figure(figsize=(6, 5), dpi=240)
        gs = GridSpec(3, 2, width_ratios=(3, 1), wspace=0.2, hspace=0.55)
        
        axs = [ f.add_subplot(gs[:, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[1, 1]), f.add_subplot(gs[2, 1])]

        # Query the GAIA database for bright stars
        ra, dec = design.raBoresight, design.decBoresight
        r = GaiaReader(
        limits = { 'gaia_g': [ -10, 16 ] }
        )

        gaia = r.cone_search((ra, dec), 50)
        print(gaia.shape)

        # Conver GAIA star positions into focal plane positions
        pointing = Pointing(design.raBoresight, design.decBoresight, posang=design.posAng,
                        obs_time=config.field.obs_time)

        parallax = np.array(gaia.data['parallax'])
        parallax[parallax < 1e-7] = 1e-7

        gaia_fp_pos = pfi.radec_to_fp_pos(
                *gaia.get_coords(),
                pointing=pointing,
                epoch=2016.0,
                pmra=np.array(gaia.data['pmra']),
                pmdec=np.array(gaia.data['pmdec']),
                parallax=np.array(gaia.data['parallax']),
                rv=np.array(gaia.data['rv'])
                )

        # MAGNITUDE DISTRIBUTION OF THE TARGETS

        ax = axs[1]

        for j, target_type in enumerate([TargetType.SCIENCE, TargetType.SKY, TargetType.FLUXSTD]):
                idx = np.where((design.fiberStatus == FiberStatus.GOOD) & (design.targetType == target_type))[0]
                flux = np.array([ design.fiberFlux[i][0] for i in idx if len(design.fiberFlux[i]) > 0 ])
                mag = nJy_to_ABmag(flux)
                
                ax.hist(mag, color=cmap(j), label=target_type)
                
        ax.set_xlim(13.1, 24.9)

        ax.set_xlabel('mag')            # Filter varies by source
        ax.set_ylabel('count')

        ax.set_title('Science targets and flux std')

        # MAGNITUDE DISTRIBUTION OF THE BRIGHT POINT SOURCES

        ax = axs[2]

        m, _ = gaia.get_magnitude(gaia.photometry['gaia'].magnitudes['g'])
        ax.hist(m)

        ax.set_xlabel('GAIA mag g')
        ax.set_ylabel('count')
        ax.set_title('Bright stars')


        # DISTRIBUTION OF FIBER DISTANCE FROM BRIGHT SOURCES

        ax = axs[3]

        # Ignore fibers without cobras
        mask = pfi.fiber_map.cobraId[design.fiberId - 1] != 65535

        d = distance_matrix(design.pfiNominal[mask], np.stack([gaia_fp_pos.real, gaia_fp_pos.imag], axis=-1))
        d.shape

        bins = np.linspace(0, 10, 20)
        hist, bins = np.histogram(d.ravel(), bins=bins)

        ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

        ax.set_xlabel('distance to bright star [mm]')
        ax.set_ylabel('fiber count')


        # PLOT THE COBRAS
        ax = axs[0]

        ms = 6

        # Broken fibers
        mask = (design.fiberStatus == FiberStatus.BROKENFIBER)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='x', s=ms, label='broken',
                color='black', edgecolors='none', linewidth=0.5)

        # Draw circle around broken cobras with working fibers
        mask = (design.fiberStatus == FiberStatus.BROKENCOBRA) & (design.fiberStatus != FiberStatus.BROKENFIBER)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=16, label='broken',
                color='none', edgecolors='k', linewidth=0.5)

        # Unassigned fibers
        mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.UNASSIGNED)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='unassigned',
                color='none', edgecolors='black', linewidth=0.5)

        # Good science targets
        mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.SCIENCE)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='science',
                color='green', edgecolors='none')

        # Good sky fibers
        mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.SKY)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='sky',
                color='cyan', edgecolors='none')

        # Good flux standards
        mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.FLUXSTD)
        ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='fluxstd',
                color='red', edgecolors='none')

        # Cobra centers
        ax.scatter(pfi.bench.cobras.centers.real, pfi.bench.cobras.centers.imag, 
                s=0.2, color='black', edgecolors='none')


        # Bright stars
        ax.scatter(gaia_fp_pos.real, gaia_fp_pos.imag, marker='o', s=10, label='bright',
                color='none', edgecolors='red')

        ax.set_xlabel('x [mm]')
        ax.set_ylabel('y [mm]')

        ax.set_title(
                f'designId=0x{pfsDesignId:016x} ({design.raBoresight:0.02f}, {design.decBoresight:0.02f}, PA={design.posAng:0.01f})\n'
                f'{design.designName}')

        ax.set_aspect('equal', adjustable='datalim')

        # Make legend horizontal
        ax.legend(loc='upper right', ncol=2, fontsize='small')

        # f.tight_layout()

In [0]:
# Unique catIds
for pfsDesignId, design in designs.items():
    print(f'0x{pfsDesignId:016x}', np.unique(design.catId))

In [0]:
# Unique proposalIds
for pfsDesignId, design in designs.items():
    print(f'0x{pfsDesignId:016x}', np.unique(design.catId))

In [0]:
# objId ranges
for pfsDesignId, design in designs.items():
    for target_type in [TargetType.SCIENCE, TargetType.SKY, TargetType.FLUXSTD]:
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == target_type)
        if mask.sum() > 0:
            print(f'0x{pfsDesignId:016x}', target_type, design.objId[mask].min(), design.objId[mask].max())
        else:
            print(f'0x{pfsDesignId:016x}', target_type, 'no such targets in design')